# Transfer Learning

Transfer learning is the technique of finding a model out there similar to our problem and fine tune it for our data.

Let's see if transfer learning performs better than the models we've build earlier for Food101.

## Setup

We've written some modules in *modular* directory. Let's leverage them.

In [1]:
# For this notebook to run with updated APIs, we need torch 1.12+ and torchvision 0.13+
try:
    import torch
    import torchvision
    assert int(torch.__version__.split(".")[1]) >= 12, "torch version should be 1.12+"
    assert int(torchvision.__version__.split(".")[1]) >= 13, "torchvision version should be 0.13+"
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")
except:
    print(f"[INFO] torch/torchvision versions not as required, installing nightly versions.")
    !pip3 install -U torch torchvision torchaudio --extra-index-url https://download.pytorch.org/whl/cu113
    import torch
    import torchvision
    print(f"torch version: {torch.__version__}")
    print(f"torchvision version: {torchvision.__version__}")

torch version: 1.12.1
torchvision version: 0.13.1


In [6]:
# Continue with regular imports
import matplotlib.pyplot as plt
import torch
import torchvision

from torch import nn
from torchvision import transforms

# Try to get torchinfo, install it if it doesn't work
try:
    from torchinfo import summary
except:
    print("[INFO] Couldn't find torchinfo... installing it.")
    !pip install -q torchinfo
    from torchinfo import summary

# Try to import the going_modular directory, download it from GitHub if it doesn't work
try:
    from modular.src.data import data_setup
    from modular.src.data import get_data
except:
    # Get the going_modular scripts
    print("[INFO] Couldn't find going_modular scripts... downloading them from GitHub.")
    !git clone https://github.com/JpChii/pytorch.git
    !mv pytorch/modular .
    !rm -rf pytorch
    from modular.src.data import data_setup
    from modular.src.data import get_data

## Get data

Let's download the data before we start transfer learning

In [7]:
images_dir = get_data.get_data(
    request_url="https://github.com/mrdbourke/pytorch-deep-learning/raw/main/data/pizza_steak_sushi.zip",
    data_path="data/"
)

Image download directory: data/pizza_steak_sushi
Zip path: data/pizza_steak_sushi.zip
data directory exists
data/pizza_steak_sushi directory exists
Unzip data
data/pizza_steak_sushi.zip cleand after unzip


In [8]:
# Setup train and test directories
from pathlib import Path
train_dir = Path(f"data/{images_dir}/train")
test_dir = Path(f"data/{images_dir}/test")
train_dir, test_dir

(PosixPath('data/pizza_steak_sushi/train'),
 PosixPath('data/pizza_steak_sushi/test'))

## Creating Datasets and DataLoaders

We'll use our modular imports to create datasets and dataloaders.

> Note: As of torchvision v0.13+ there's an update on how transforms can be create using `torchvision.models`. The previous method is manual and current method is auto.

While using pretrained models, we've to make sure the custom data is in the same form as the original data used to train the pretrained model.

Prior to torchvision v0.13+, to create a transform for pretrained model in `torchvision.models`.
The documentation stated below:

> All pre-trained model expect input images normalized in the same way. i.e mini-batches of 3-channel RGB images of shape (3 * H * W) where H and W expected to be at least 224.

> The images have to be loaded in to a range if [0, 1] and then normalized using mean=[0485, 0.456, 0.406] and std=[0.229, 0.224, 0..225].

### Manual transform

In [9]:
# Creating manual transform
manual_transforms = transforms.Compose([
    transforms.Resize(size=(224, 224)), # Resize images to 224, 224
    transforms.ToTensor(), # Convert to tensor and between [0, 1]
    transforms.Normalize(
        mean=[0.485, 0.456, 0.406],
        std=[0.229, 0.224, 0.225],
    )
])
manual_transforms

Compose(
    Resize(size=(224, 224), interpolation=bilinear, max_size=None, antialias=None)
    ToTensor()
    Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
)

In [ ]:
# Let's create the dataloder